<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S75_drone_decision_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q -U gtbook


In [ ]:
import numpy as np
import plotly
import plotly.subplots as pls
import plotly.graph_objects as go
import plotly.express as px

import gtsam
from gtsam import Point2

from gtbook import diffdrive
from gtbook.discrete import Variables
from gtbook.display import show, pretty

import torch
DEVICE = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

rng = np.random.default_rng()


In [ ]:
def create_random_map(W, H, num_obstacles=50, seed=42):
    """
    Creates a random occupancy map with the given dimensions and number of obstacles.
    Optionally, a seed can be provided to make the map reproducible.
    """
    # Set the seed for reproducibility:
    rng = np.random.default_rng(seed=seed)

    # Map dimensions in pixels (10 cm per pixel)
    pixel_W, pixel_H = 10*W, 10*H

    # Initialize an empty occupancy map
    obstacles = torch.zeros((pixel_H, pixel_W), dtype=torch.float32)

    # Add obstacles
    for _ in range(num_obstacles):
        # Random position for the obstacle
        x, y = rng.integers(0, pixel_W), rng.integers(0, pixel_H)

        # Random value for the obstacle (higher than empty space)
        obstacle_value = rng.random() * 0.5 + 0.5  # Random value between 0.5 and 1.0

        # Assign the value to the obstacle position
        obstacles[y, x] = obstacle_value

    # Define a box filter (kernel)
    kernel_size = 5  # The size of the box filter
    kernel = torch.ones((kernel_size, kernel_size), dtype=torch.float32)

    # Apply 2D convolution to "fatten" the objects
    batch = obstacles[None, None, ...]  # Add batch and channel dimensions
    return torch.conv2d(batch, kernel[None, None, ...], padding='same')[0, 0, ...]

In [ ]:
def displaced_gaussian(
    sigma: float, K: int, uv: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns a Gaussian kernel of size K x K with standard deviation sigma.
    The kernel is centered at uv, a float coordinate in xy convention.
    Also returns the indices of the kernel in the image.
    """
    # Create meshgrid around uv, in ij convention
    j0, i0 = np.round(uv).astype(int)
    kk = np.arange(-(K // 2), K // 2 + 1)
    ii, jj = np.meshgrid(i0 + kk, j0 + kk, indexing="ij")

    # Calculate the kernel
    dd = (ii - uv[1]) ** 2 + (jj - uv[0]) ** 2
    kernel = np.exp(-dd / (2 * sigma**2))
    kernel = kernel / kernel.sum()

    # Return the kernel and the indices of the kernel in the image
    return kernel, ii, jj


def gaussian_kernel(sigma: float, K: int = 9) -> torch.Tensor:
    """Generates a Gaussian kernel of shape (1, 1, K, K) with standard deviation sigma."""
    uv = np.array([K // 2, K // 2])
    numpy_kernel, _, _ = displaced_gaussian(sigma, K, uv)
    return torch.tensor(numpy_kernel, dtype=torch.float32)[None, None, ...]


def gaussian_filter(sigma, uv, image, K=9):
    """Applies a Gaussian filter at uv on the given image."""
    kernel, ii, jj = displaced_gaussian(sigma, K, uv)
    
    # Dimensions of the image
    height, width = image.shape[-2:]

    # Find the valid range for ii and jj
    valid_mask = (ii >= 0) & (ii < height) & (jj >= 0) & (jj < width)
    valid_ii = ii[valid_mask]
    valid_jj = jj[valid_mask]

    # Adjust the kernel to match the valid range
    valid_kernel = kernel[valid_mask]

    # Extract the relevant portion of the image
    image_excerpt = image[..., valid_ii, valid_jj].numpy()

    # Apply the valid kernel to the image excerpt
    return (valid_kernel * image_excerpt).sum(axis=-1)

In [ ]:
def sobel_kernels(dtype=torch.float32, **kwargs):
    """Return Sobel gradient kernels sobel_u of shape (1, 1, 1, 3) and sobel_v of shape (1, 1, 3, 1)."""
    sobel_u = torch.tensor([[-1, 0, 1]], dtype=dtype, **kwargs)
    sobel_v = sobel_u.T
    return sobel_u[None, None, ...], sobel_v[None, None, ...]

In [ ]:
class SmoothTrajectory:
    def __init__(self, trajectory: gtsam.Values, N=5):
        """
        Construct a polynomial fit of degree N-1, e.g. N=4 is a cubic.
        Works by resampling the trajectory at the Chebyshev nodes of the second kind.
        """
        self.N = N
        xy = gtsam.utilities.extractPoint2(trajectory)

        # Calculate N-2 Chebyshev nodes of the second kind in the [0,K-1] interval,
        # and interpolate points at these Chebyshev nodes:
        K = len(xy)
        nodes = [gtsam.Chebyshev2.Point(N, i, 0, K-1) for i in range(1, N - 1)]
        middle = [xy[int(t)] * (int(t) + 1 - t) + xy[int(t) + 1] * (t - int(t)) for t in nodes]

        # Combine start point, middle points, and end point:
        self.points = np.stack([xy[0], *middle, xy[-1]], axis=0)

    def evaluate(self, t: np.ndarray, a=0, b=1):
        """Evaluate the polynomial at t, which should a 1D array."""
        weights = gtsam.Chebyshev2.WeightMatrix(self.N, t, a, b)
        return weights @ self.points

    def velocities(self, t: np.ndarray, a=0, b=1):
        """Evaluate the polynomial at t, which should a 1D array."""
        D = gtsam.Chebyshev2.DifferentiationMatrix(self.N, a, b)
        derivatives = D @ self.points
        weights = gtsam.Chebyshev2.WeightMatrix(self.N, t, a, b)
        return weights @ derivatives

    def accelerations(self, t: np.ndarray, a=0, b=1):
        """Evaluate the polynomial at t, which should a 1D array."""
        D = gtsam.Chebyshev2.DifferentiationMatrix(self.N, a, b)
        DD = D @ D
        derivatives = DD @ self.points
        weights = gtsam.Chebyshev2.WeightMatrix(self.N, t, a, b)
        return weights @ derivatives

# Trajectory Optimization

> We can use factor graphs to optimize over future trajectories, as well.

<img src="Figures7/S75-Autonomous_camera_drone-05.jpg"  alt="Splash image with steampunk drone in aggressive maneuver" width="40%" align=center style="vertical-align:middle;margin:10px 0px">


In the previous section we saw how do use factor graphs for visual SLAM and structure from motion. These perception algorithms are typically run after we have gathered some visual information, and inform us about what happened in the past. But how can we plan for the future? We already saw that RRTs are useful paradigm to plan in a continuous, potentially high dimensional state space. 

In this section we will discuss a different method, which is optimal control. In particular, we will use **trajectory optimization**, to minimize the cost associated with the trajectory that we would like to execute in the future. These costs can be associated with staying away from obstacles, or other desirable properties like minimizing power consumption to preserve battery life. Finally, RRTs and trajectory optimization can be combined, where RRT finds a good "broad strokes" trajectory, and trajectory optimization smooths out and fine-tunes the final trajectory.

## A Simple Objective

We explain trajectory optimization using a very simple problem: how to get from point A to point B. This seems like an easy problem, as we all know the answer should be a straight line, right? But, it is not as easy for drones, as we have to account for the drone's attitude, and we might have to accelerate and decelerate at the start and end points. Come to think of it, maybe we are already flying at a certain velocity at point A, and want to have a certain velocity at point B! Clearly, this is not as easy as it appears.

There are many ways to go about this:
- MPC with collocated polynomials
- discrete poses and controls + dynamics factors
- differential flatness + controller: this is what we'll do

## Optimizing for Position

It turns out we can optimize for position only, without worrying about attitude, at least at first. This is not a trivial statement, and it is also very fortunate. Let us assume that we want to fly in a forest, full of trees, and we want a trajectory for the drone avoid the trees. A quadrotor-style drone trajectory has the potential to be rather complicated: the drone lives in the 6DOF space of position (3DOF) and attitude (3DOF). And we know from Section 7.2 that attitude will *affect* position: if we pitch or role, that will affect our velocity, and that will integrate into position. That very mechanism is also a source of complexity: a drone is *under-actuated*: we cannot directly control the 6DOF, we have to do it through the action only four rotors. It seems like a very complex problem indeed.

When given a continuous trajectory of position that also behaves well in terms of velocity, we can obtain the pitch, roll, and thrust trajectories to obtain it. Of course, it can't be a totally crazy trajectory: a drone can't stop on a dime, or teleport, or go faster than certain limits. But if we have a 3DOF translational trajectory that is well behaved, we can "upgrade" it to a full position/attitude in two different ways: in advance, by solving a large optimization problem, or physically, in real-time, by using a tracking controller. The latter is what we will do, after obtaining a trajectory.

To find a suitable translational trajectory that avoids obstacles, we will use optimization. There are many ways one can do this, some better than others, but here strive for simplicity: we discretize the time interval in which we expect to fly and solve for a set of corresponding set of positions over time that satisfy our objectives. These objectives can include:

- a desired starting position (and velocity, if so desired);
- a desired goal position (and velocity, if desired);
- a desired distance from the closest obstacle at any time;
- a smoothness objective that bounds our velocity and/or acceleration.

If these objectives are in conflict, we can assign them weights such that we trade off some against others. Constrained optimization techniques allow to set hard constraints and/or bounds, but we do not use those here. Instead, we opt for a simple non-linear least squares scheme that minimizes the following objective:

$$
X^* = \arg \min \sum_{k=1}^K \phi_k(X_k)^2  + \sum_{k=1}^{K-1} \psi_k(X_k, X_{k+1})^2
$$

where $X_k$ is the position at time, and $\phi_k(X_k)$ and $\psi_k(X_k, X_{k+1})$ are unary and binary objectives, respectively, that we want to minimize. Desired start and goal, as well as obstacle avoidance are examples of unary objectives, while smoothness is a binary objective, as it evaluates the distance/velocity between successive positions.

It will not come as any surprise that we can also use factor graph optimization to find our optimal trajectories. The objective above is exactly the expression of a nonlinear factor graph, except that the factors now derive from objectives we want to minimize, rather than measurement errors. In the next section we work in detail how to do so.

## Occupancy and Cost Maps

A very popular approach to represent environments with different obstacles is to use an **occupancy map**. An occupancy map is essentially a 2D array of cells, each representing the environment at a certain resolution (for example, 10 cm by 10 cm). Each cell in this map contains a probability for the presence of an obstacle. This concept is further extendable to distinguish between various types of obstacles, each with different levels of lethality. This extended version can then be transformed into a **cost map**, where areas with less hazardous obstacles (like tall grass) are considered less costly compared to more dangerous ones (like flying into a tree). For our purposes, we will start directly with cost maps instead of creating true occupancy maps.

To illustrate, let us consider creating a simple example cost map. In the `gtbook` library, we have a function named `create_random_map`. This function requires as parameters the width $W$, height $H$, and the number of obstacles to create a random arrangement of obstacles within the map. Width and height are given in meters, and the resolution is hardcoded at 10 cm. An optional parameter is a random seed, which allows for the reproduction of the exact experiment setup. The following code snippet demonstrates how to generate a random map with 50 obstacles using a random seed of 42:

In [ ]:
# Create a random cost map
W, H = 30, 10  # 30m x 10m
cost_map = create_random_map(W, H, num_obstacles=50, seed=7)
px.imshow(cost_map, color_continuous_scale='Reds').show()
#| caption: Cost map with randomly generated obstacles, at 10cm resolution.
#| label: fig:obstacles

To create differentiable factors $\phi_k(X_k)$ that avoid high cost areas, we can smooth the cost map with a differentiable kernel, like a Gaussian. The cost map above is a torch tensor, representing a grayscale image 100 pixels tall and 300 pixels wide. Convolving an image with a kernel is relatively easy in pytorch, though we need to juggle some tensor dimensions because by convention, `conv2d` operates on $\text{batch} \times \text{channel} \times \text{height} \times \text{width}$ tensors:

In [ ]:
sigma = 0.5  # 0.5m standard deviation for the Gaussian kernel
K = 21 # 21x21 kernel is big enough to accommodate that standard deviation 
kernel = gaussian_kernel(sigma*10, K) # multiply by 10 as map is 10cm resolution
batch = cost_map[None, None, ...]  # Add batch and channel dimensions
blurred = torch.conv2d(batch, kernel, padding='same')[0, 0, ...]
px.imshow(blurred, color_continuous_scale='Reds').show()
#| caption: Cost map blurred with 0.5 meter Gaussian kernel.
#| label: fig:blurred

To get truly differentiable factors, however, it is better to construct a custom Gaussian kernel at the exact location we want to evaluate the cost. Remember we are trying to minimize a trajectory, by moving around a set of positions $X_k$ at discrete time stamps $t_k$. In non-linear optimization, the story is always the same: evaluate the cost *and* its derivatives at a current guess, and then use the derivatives to move to a lower cost solution. However, if we smooth a discretized cost map, the result is still discretized, however smooth it looks to us. To get the *exact* value of the cost, and its derivatives, we can create a Gaussian kernel centered *exactly* at our current *continuous* position.

The function `gaussian_filter` does just that: it evaluates a single Gaussian kernel operator given a standard deviation, a continuous map location, and the map to operate on. The code below shows that this gives the exact same result as a lookup in the blurred image when given an integer coordinate, but it *also* works for non-integer coordinates:

In [ ]:
xy = Point2(7.5, 5.3) # point in map, in meters
uv = 10*xy # continuous position in image
local_result = gaussian_filter(sigma*10, uv, cost_map, K)
print(f"Local cost at {xy} is {local_result:.3f}")

# When uv are at integer values, blurred image gives the same result:
assert np.allclose(local_result, blurred[int(uv[1]), int(uv[0])])

To get the derivatives we blur the vertical and horizontal gradient images, respectively. Remember that we need the derivative of the exact cost with respect to x and y, if we are to optimize over those positions. Because convolution is a linear operation, we can take the derivative of the Gaussian kernel with respect to x and y, *or* we can apply the convolution on a derivative image. We choose the latter, by first using the Sobel operator we encountered in Section 5.4 to create gradient images: 

In [ ]:
# Compute gradients:
sobel_u, sobel_v = sobel_kernels()
grad_u = torch.conv2d(batch, sobel_u, padding='same')[0,0,...]
grad_v = torch.conv2d(batch, sobel_v, padding='same')[0,0,...]

In [ ]:
#| caption: Obstacle cost map, blurred cost map, and both vertical and horizontal sobel gradients.
#| label: fig:obstacles-2x2
fig = pls.make_subplots(rows=2, cols=2)
fig.add_trace(go.Heatmap(z=cost_map, colorscale='reds', showscale=False), row=1, col=1)
fig.add_trace(go.Heatmap(z=blurred, colorscale='reds', showscale=False), row=1, col=2)
fig.add_trace(go.Heatmap(z=grad_u, colorscale='rdbu', showscale=False), row=2, col=1)
fig.add_trace(go.Heatmap(z=grad_v, colorscale='rdbu', showscale=False), row=2, col=2)
for i in [1,2]:
    for j in [1,2]:
        fig.update_xaxes(title_text="", showticklabels=False, row=i, col=j)
        fig.update_yaxes(title_text="", showticklabels=False, row=i, col=j)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.show()

Figure <a href="#fig:obstacles-2x2" data-reference-type="ref" data-reference="fig:obstacles-2x2">3</a> shows both cost map and its blurred version, as well as the (un-blurred) gradient images `grad_u` and `grad_v`. We can then combined these into one rank-3 tensor and evaluate the cost *and* derivatives in one operation:

In [ ]:
combined = torch.stack([cost_map, grad_u, grad_v], dim=0)
print(np.round(gaussian_filter(sigma*10, uv, combined, K),3))

## Factor Graphs for Trajectory Optimization

Now that we can evaluate cost its derivatives at any location, we can create factors. Since occupancy maps or cost maps are not built into GTSAM, we use its facility to create a custom factor from arbitrary python code. A `gtsam.CustomFactor` just has a constructor that can take an arbitrary python callback function, along with a `Key` (and a noise model). At evaluation, the callback function gets a handle to the factor and a `gtsam.Values` object, and it is supposed to do three things:

- check which variable is involved, by asking the factor;
- with that key, extract the current estimate from the passed in `Values`;
- calculate the cost, and if asked, its derivatives.

The fact that we can do this for arbitrary python code is very convenient, and is used below to create cost factors that interrogate the cost map, using our localized Gaussian filter, and if asked, its derivatives:

In [ ]:
def error_func(factor: gtsam.CustomFactor, v: gtsam.Values, H: list[np.ndarray]):
    """Error function for the custom factor."""
    # Extract the Point2 variable using the key in the factor
    point = v.atPoint2(factor.keys()[0])

    uv = 10 * point # Convert to map coordinates

    if H is None:
        cost = gaussian_filter(sigma * 10, uv, cost_map, K)
    else:
        # If Jacobian is needed, calculate them here:
        cost, cost_x, cost_y = gaussian_filter(sigma * 10, uv, combined, K)
        H[0] = np.array([[cost_x, cost_y]])

    return np.array([cost]).astype(float)

Let us discretize the trajectory with M factors, using the keys $k\in 1...M$. We can define a start and goal point, and initialize the trajectory as straight line. Who knows, we might be lucky and hit no obstacles along the way:

In [ ]:
M = 100 # Number of points
start, goal = Point2(2, 5), Point2(28, 5)
initial = gtsam.Values()
for k in range(1,M+1):
    alpha = (k-1)/(M-1)
    initial.insert(k, (1-alpha)*start + alpha*goal)

In [ ]:
#| caption: Initial trajectory estimate plotted on the cost map.
#| label: fig:initial
initial_path = 10*gtsam.utilities.extractPoint2(initial)
fig = px.imshow(cost_map, color_continuous_scale='Reds')
fig.add_trace(go.Scatter(x=initial_path[:,0], y=initial_path[:,1], mode='lines+markers', line=dict(color='gray')))
fig.update_layout(coloraxis_showscale=False, showlegend=False, margin=dict(l=10, r=10, t=10, b=10))

Plotting this trajectory on the cost map shows that we are not lucky, however: we are definitely going through some obstacles. We can immediately create all obstacle cost factors now, and evaluate the cost to conform this is not a zero-cost path. Since these factors are non-linear, we store them in a `gtsam.NonlinearFactorGraph`:

In [ ]:
obstacle_factors = gtsam.NonlinearFactorGraph()
for k in range(1,M+1):
    custom_factor = gtsam.CustomFactor(None, [k], error_func)
    obstacle_factors.add(custom_factor)
print(f"Graph has {obstacle_factors.size()} factors")

The error profile along the trajectory clearly corresponds with the obstacles we encounter along the way. We can evaluate both the individual errors and the sum-of-squares error $\sum\psi_k(X_k)^2$ of a factor graph. The former gives us an error profile along the way, and the latter represents a loss we would like to minimize. We show the former as a bar graph below:

In [ ]:
#| caption: Errors for the initial trajectory estimate.
#| label: fig:initial-errors
errors = -obstacle_factors.linearize(initial).jacobian()[1]
fig = px.bar(errors, labels={'x': 'Factor', 'y': 'Error'}, range_y=[0, 0.2])
fig.update_layout(coloraxis_showscale=False, showlegend=False, height=300, margin=dict(l=10, r=10, t=10, b=10))

A complete trajectory optimization problem needs more than just obstacle factors, however. We also need to add start and goal factors, as well as smoothness factors. Start and goal factors make sure that we start at our current position, and tell our optimizer where we want to go. They can both be implemented as simple priors on the variables with keys $k=1$ and $k=100$. Smoothness factors are implemented using a `gtsam.BetweenFactor`, and make sure that we obtain a smooth, continuous trajectory.

Both types of factors are added in the code below:

In [ ]:
# We start by copying the obstacle factors:
graph = gtsam.NonlinearFactorGraph()
graph.push_back(obstacle_factors)

constrained2 = gtsam.noiseModel.Constrained.All(2) # We *insist* on the start and goal points!
graph.addPriorPoint2(1, start, constrained2)
graph.addPriorPoint2(M, goal, constrained2)

# The strength of the smoothness factors is controlled by a `precision` value: higher is stronger.
# Feel free to change the precision value below and see how the optimizer behaves:
smoothness_model = gtsam.noiseModel.Isotropic.Precision(2, 0.01)
for k in range(1, M):
    graph.add(gtsam.BetweenFactorPoint2(k, k+1, gtsam.Point2(0.0, 0.0), smoothness_model))

We can also add a hard constraint on the initial pairs of points to enforce zero velocity at the start:

In [ ]:
graph.add(gtsam.BetweenFactorPoint2(1, 2, gtsam.Point2(0.0, 0.0), constrained2))

To optimize, we use a Levenberg Marquardt non-linear optimizer. This optimizer, which we have encountered before, linearizes the factors at every iteration and takes a cautious second-order minimization step. The "cautiousness" is governed by a parameter $\lambda$ that essentially selects between gradient descent ($\lambda$ high) and a second-order Gauss-Newton step ($\lambda$ low).  In our potentially very non-linear cost landscape it is good practice to start of the optimizer with a relatively high value for $\lambda$:

In [ ]:
print(f"Initial error: {graph.error(initial):.3f}")
params = gtsam.LevenbergMarquardtParams()
params.setlambdaInitial(100)
# params.setVerbosityLM("SUMMARY") # uncomment this line to see the LM progress
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)
result = optimizer.optimize()
print(f"Final error: {graph.error(result):.3f}")

The optimizer is very fast, on the order of milliseconds, so it can be used in an online planner that continuously re-plans as the cost map is updated with new information. When used for flying a drone, we can execute on the initial part of the trajectory and then immediately re-plan. This scheme is known as "model-predictive" control, and is frequently used in actual autonomous vehicles and drones.

In our case, the optimizer found a very nice trajectory that navigates some tricky obstacle configurations, especially in the second half of the trajectory:

In [ ]:
#| caption: Initial and final trajectories plotted on the blurred cost map.
#| label: fig:final
result_path = 10*gtsam.utilities.extractPoint2(result)
fig = px.imshow(blurred, color_continuous_scale='Reds')
fig.add_trace(go.Scatter(x=initial_path[:,0], y=initial_path[:,1], mode='lines+markers', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=result_path[:,0], y=result_path[:,1], mode='lines+markers', line=dict(color='green')))
fig.update_layout(coloraxis_showscale=False, showlegend=False, margin=dict(l=10, r=10, t=10, b=10))

The error profile along this final trajectory is not entirely down to zero, as we have to navigate some tight obstacle configurations along the way. However, when plotted on the same scale it looks much better than for the initial trajectory:

In [ ]:
#| caption: Errors for the final trajectory result.
#| label: fig:final-errors
errors = -obstacle_factors.linearize(result).jacobian()[1]
fig = px.bar(errors, labels={'x': 'Factor', 'y': 'Error'}, range_y=[0, 0.2])
fig.update_layout(coloraxis_showscale=False, showlegend=False, height=300, margin=dict(l=10, r=10, t=10, b=10))

## Smoothing

- A magic Chebyshev interpolation class that gets us derivatives and accelerations
- Can also be used for RRT, see project

In [ ]:
smooth = SmoothTrajectory(result, N=20)
T = 20.0
t = np.linspace(0.0 , T, 250)
smoothed = smooth.evaluate(t, 0, T)
velocities = smooth.velocities(t, 0, T)
accelerations = smooth.accelerations(t, 0, T)

In [ ]:
#| caption: Discretized and smoothed trajectory.
#| label: fig:smoothed
fig = px.imshow(blurred, color_continuous_scale='Reds')
fig.add_trace(go.Scatter(x=result_path[:,0], y=result_path[:,1], mode='lines', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=10*smoothed[:,0], y=10*smoothed[:,1], mode='lines', line=dict(color='green')))
fig.add_trace(go.Scatter(x=10*smooth.points[:,0], y=10*smooth.points[:,1], mode='markers', line=dict(color='green')))
fig.update_layout(coloraxis_showscale=False, showlegend=False, margin=dict(l=10, r=10, t=10, b=10))

In [ ]:
#| caption: smoothed trajectory as time series.
#| label: fig:smoothed-series
# Create 4x1 subplots and plot x, dx, y, dy:
fig = pls.make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=t, y=velocities[:,0], mode='lines', line=dict(color='gray'), name='vx'), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=accelerations[:,0], mode='lines', line=dict(color='gray'), name='ax'), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=velocities[:,1], mode='lines', line=dict(color='gray'), name='vy'), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=accelerations[:,1], mode='lines', line=dict(color='gray'), name='ay'), row=4, col=1)
fig.update_layout(coloraxis_showscale=False, showlegend=False, margin=dict(l=80, r=60, t=10, b=10))


## Translational Control

- A thrust controller
- A pitch/roll controller
  - calculate desired acceleration
  - from that we calculate thrust vector we need, compensate for gravity
  - project onto the F and L body axes

## Planning and Controlling Yaw

- We'd like to point in a particular direction
- forward looking, or camera drone
- Explain why controlling yaw is harder
- A simple yaw controller